In [12]:
# 導入必要條件
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_ollama import ChatOllama

# 建立 Ollama 模型
model = ChatOllama(
    model="gpt-oss:20b",
    temperatrue=0.7
    )

In [13]:
# 定義主要提示模板 - 餐廳評論分析
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","你是一個專業的餐廳評論家，擅長分析餐廳的優缺點。"),
        ("human","請分析這家餐廳 : {restaurant_info}"),
    ]
)

In [14]:
pros_template = ChatPromptTemplate.from_messages(
    [
        ("system","你是一個專業的餐廳評論家，擅長分析餐廳的優點。"),
        ("human","根據餐廳資訊 : {restaurant_info}，請列出這家餐廳的優點。"),
    ]
)

pros_branch_chain =  (
    pros_template
    | model
    | StrOutputParser()
)

In [15]:
cons_template = ChatPromptTemplate.from_messages(
    [
        ("system","你是一個專業的餐廳評論家，擅長分析餐廳的缺點。"),
        ("human","根據餐廳資訊 : {restaurant_info}，請列出這家餐廳的缺點。"),
    ]
)

cons_branch_chain =  (
    cons_template
    | model
    | StrOutputParser()
)

In [16]:
def combine_pros_cons(pros, cons):
    """將優點和缺點合併成最終評論"""
    return f"""
🍽️ 餐廳評論分析報告
{'='*50}

✅ 優點分析：
{pros}

❌ 缺點分析：
{cons}

📊 總結：這是一個平衡的餐廳評論分析
"""

In [17]:
chain = (
    prompt_template 
    | model 
    | StrOutputParser()
    | RunnableLambda(lambda x:{"restaurant_info":x})
    | RunnableParallel(
        pros = pros_branch_chain,
        cons = cons_branch_chain
    )
    | RunnableLambda(lambda x: combine_pros_cons(x['pros'],x['cons']))
)

In [18]:
restaurant_info = """
這是一家位於台北市中心的義大利餐廳，提供傳統義式料理。
餐廳環境優雅，服務人員親切，但價格偏高，等待時間較長。
食物品質不錯，但份量偏少。適合約會或特殊場合用餐。
"""

result = chain.invoke({"restaurant_info": restaurant_info})

print(result)


🍽️ 餐廳評論分析報告

✅ 優點分析：
### 本家義大利餐廳的主要優點

| 類別 | 具體優點 |
|------|----------|
| **環境與氛圍** | - 以歐式復古裝潢營造沉浸式義大利氛圍，燈光柔和、音樂恰到好處，打造浪漫、適合約會與特殊場合的場景。<br>- 桌椅距離適中，私密度高，特別適合兩人對談或小型聚會。<br>- 位於台北市中心，交通便利，能夠吸引多樣客群。 |
| **服務品質** | - 員工禮貌、專業，能即時回應客人需求。<br>- 服務人員對菜品與酒水擁有豐富知識，對不熟悉義式料理的顧客耐心解說。<br>- 服務流程流暢，能在高峰期仍保持良好溝通效率。 |
| **料理品質** | - 以傳統義式料理為主，醬汁平衡、香料運用恰到好處，麵條口感優秀。<br>- 采用新鮮本地食材與義大利進口食材結合，保證食材新鮮度與風味。<br>- 部分菜品加入台灣在地食材，帶來微妙的異國風味創新。<br>- 份量雖偏小，但非常適合共享或搭配多道小盤，增添用餐樂趣。 |
| **價格與價值** | - 雖定位高檔，主菜價格約 1,200–2,000 元，但以環境、料理與服務相加仍屬合理範圍。<br>- 適合想要一次性提升餐飲體驗、慶祝特殊日子（如周年紀念、生日、商務宴請）的顧客。 |
| **特殊場合適用性** | - 既能作為浪漫約會的絕佳選擇，也適合商務宴請、生日慶祝、周年紀念等特殊場合。 |
| **總體評價** | - 綜合環境、服務與料理三大要素，給予 4.0/5 的總評，展現其在高品質餐飲體驗上的領先地位。 |

> **結語**：若您願意為一場高品質、充滿浪漫氛圍的義大利餐飲體驗付出額外費用，並能忍受一定的等待時間，這家餐廳將是您理想的選擇。祝您用餐愉快！

❌ 缺點分析：
**缺點總覽**

1. **空間狹小且擁擠**  
   - 高地段、客流量大，週末及節假日特別擁擠，影響用餐舒適度。

2. **上菜速度偶爾延遲**  
   - 人手短缺時，特別在點餐高峰期，菜品上桌時間較長。

3. **份量偏小**  
   - 主菜多為小份量，適合兩人共享；若想要主食份量，需點多道小盤或追加。

4. **價格偏高**  
   - 主菜在 1,200–2,000 元，對預算有限的顧客或大團體而言不夠經濟。

5. **等待時